In [15]:
# load libraries
library(dplyr)
library(lme4)
#install.packages('Hmisc')
#library(Hmisc)
library(tibble)
install.packages('ciTools')
library(ciTools)

# load data
data <- read.csv('all_features_meanband_NEW.csv')

# drop rows for incomplete or no-seizure data
data <- data[data$patients != 'AAA' & data$patients != 'BBB' & data$patients !='CCC',]

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [14]:
# exclude XXX
exclude_XXX <- FALSE
if(exclude_XXX){
    data <- data[data$patients != 'XXX',]} else {
    data <- data
}

In [16]:
# remove dots from names
names(data) <- gsub("\\.", "_", names(data))
# remove empty QIDS question #6
drops <- c('X6__Mood__irritable_')
data <- data[ , !(names(data) %in% drops)]

In [17]:
# convert ratings from char format to numeric
data[, c(12:53)] <- sapply(data[,c(12:53)], as.character)
data[, c(12:53)] <- sapply(data[,c(12:53)], as.numeric)

Warning message in lapply(X = X, FUN = FUN, ...):
“NAs introduced by coercion”Warning message in lapply(X = X, FUN = FUN, ...):
“NAs introduced by coercion”Warning message in lapply(X = X, FUN = FUN, ...):
“NAs introduced by coercion”Warning message in lapply(X = X, FUN = FUN, ...):
“NAs introduced by coercion”Warning message in lapply(X = X, FUN = FUN, ...):
“NAs introduced by coercion”Warning message in lapply(X = X, FUN = FUN, ...):
“NAs introduced by coercion”Warning message in lapply(X = X, FUN = FUN, ...):
“NAs introduced by coercion”Warning message in lapply(X = X, FUN = FUN, ...):
“NAs introduced by coercion”Warning message in lapply(X = X, FUN = FUN, ...):
“NAs introduced by coercion”Warning message in lapply(X = X, FUN = FUN, ...):
“NAs introduced by coercion”Warning message in lapply(X = X, FUN = FUN, ...):
“NAs introduced by coercion”Warning message in lapply(X = X, FUN = FUN, ...):
“NAs introduced by coercion”Warning message in lapply(X = X, FUN = FUN, ...):
“NAs introduce

In [18]:
# data check (should be 15's all across)
data[data$patients=='YYY' & data$sessions==12,]$QIDS

[1] 15 15 15 15 15 15 15 15 15 15 15

In [19]:
# means of all data
mean_data <- aggregate(data[, 7:40], list(data$patients, data$sessions, data$preposts), mean, na.rm=TRUE)
mean_data <- mean_data %>% rename(
                patients = Group.1,
                sessions = Group.2,
                preposts = Group.3)

# pre data
pre_mean_data <- mean_data %>% filter(preposts=='Pre')

# post data
post_mean_data <- mean_data %>% filter(preposts=='Post')

# diff data
# diff_mean_data <- aggregate(mean_data[,4:8], list(mean_data$patients, mean_data$sessions), diff, na.rm = TRUE)
# diff_mean_data <- diff_mean_data %>% rename(
#                           patients = Group.1,
#                           sessions = Group.2)
# diff_mean_data <- add_column(diff_mean_data, preposts=rep('Diff', 23), .after = 'sessions')
# diff_mean_data <- cbind(diff_mean_data, post_mean_data[, 9:28])

In [13]:
names(pre_mean_data)

[1] "patients"                   "sessions"                  
 [3] "preposts"                   "chans_exps"                
 [5] "offsets"                    "deltas"                    
 [7] "delta_pows"                 "alphas"                    
 [9] "alpha_pows"                 "thetas"                    
[11] "theta_pows"                 "delta_band_pow"            
[13] "alpha_band_pow"             "theta_band_pow"            
[15] "l_pow"                      "h_pow"                     
[17] "total_pow"                  "QIDS"                      
[19] "X1__Sleep_onset_insomnia"   "X2__Mid_nocturnal_insomnia"
[21] "X3__Early_morning_insomnia" "X4__Hypersomnia"           
[23] "X1_4_Highest"               "X5__Mood__sad_"            
[25] "X7__Appetite__increased_"   "X8__Appetite__decreased_"  
[27] "X9__Weight__decrease_"      "X10__Weight__increase_"    
[29] "X7_10_Highest"              "X11__Concentration"        
[31] "X12__Outlook__self_"        "X13__Suicidal_ideation"    
[33] "X14__Involvement"           "X15__Energy"               
[35] "X16__Psychomotor_slowing"   "X17__Psychomotor_agitation"
[37] "X16_17_Highest"

In [14]:
# overall model fitting and confidence intervals

formula <- 'chans_exps ~ sessions + (1|patients)'
lmf <- lmer(formula, pre_mean_data, REML=FALSE)
pred <- add_ci(data.frame(sessions = 0:12), lmf,
               type='parametric', alpha=0.05, includeRanef=FALSE) #type='parametric'
write.csv(pred, 'exponent_model_predictions_exclude_NEW.csv', row.names=FALSE) # change to exlude XXX

In [15]:
formula <- 'QIDS ~ sessions + (1|patients)'
lmf <- lmer(formula, pre_mean_data, REML=FALSE)
pred_qids <- add_ci(data.frame(sessions = 0:12), lmf,
               type='parametric', alpha=0.05, includeRanef=FALSE) 

In [16]:
formula <- 'delta_band_pow ~ sessions + (1|patients)'
lmf <- lmer(formula, pre_mean_data, REML=FALSE)
pred_delta <- add_ci(data.frame(sessions = 0:12), lmf,
               type='parametric', alpha=0.05, includeRanef=FALSE) 

In [12]:
formula <- 'offsets ~ sessions + (1|patients)'
lmf <- lmer(formula, pre_mean_data, REML=FALSE)
pred_offset <- add_ci(data.frame(sessions = 0:12), lmf,
               type='parametric', alpha=0.05, includeRanef=FALSE) 

boundary (singular) fit: see ?isSingular


In [19]:
pred_delta
write.csv(pred_delta, 'delta_model_predictions_exclude_NEW.csv', row.names=FALSE)

sessions,pred,LCB0.025,UCB0.975
0,2.653172e-11,1.534595e-11,3.771750e-11
1,2.737208e-11,1.730219e-11,3.744197e-11
2,2.821243e-11,1.913058e-11,3.729428e-11
3,2.905279e-11,2.078516e-11,3.732041e-11
4,2.989314e-11,2.221047e-11,3.757581e-11
5,3.073349e-11,2.335179e-11,3.811520e-11
6,3.157385e-11,2.417439e-11,3.897331e-11
7,3.241420e-11,2.468047e-11,4.014794e-11
8,3.325456e-11,2.490797e-11,4.160115e-11
9,3.409491e-11,2.491249e-11,4.327733e-11


### loop through all features & ratings, generate csv with all slope data

In [24]:
# empty data frames to store slope data
# pre
pre_slope_data <- data.frame(matrix(ncol = 37, nrow = 10))
x <- c(names(pre_mean_data))
colnames(pre_slope_data) <- x
# post
post_slope_data <- data.frame(matrix(ncol = 37, nrow = 10))
y <- c(names(post_mean_data))
colnames(post_slope_data) <- y
# # diff
# diff_slope_data <- data.frame(matrix(ncol = 37, nrow = 10))
# z <- c(names(diff_mean_data))
# colnames(diff_slope_data) <- z

In [11]:
library(data.table)


Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last



In [22]:
names(pre_mean_nodelta)[4:16]

[1] "chans_exps"     "offsets"        "alphas"         "alpha_pows"    
 [5] "thetas"         "theta_pows"     "delta_band_pow" "alpha_band_pow"
 [9] "theta_band_pow" "l_pow"          "h_pow"          "total_pow"     
[13] "QIDS"

In [26]:
# get Pre slopes dataframe
pre_mean_nodelta = copy(pre_mean_data)
drops <- c('deltas', 'delta_pows')
pre_mean_nodelta <- pre_mean_nodelta[ , !(names(pre_mean_nodelta) %in% drops)]


for(i in names(pre_mean_nodelta)[4:16]) {
    formula = paste0(i,'~ sessions + (1 + sessions|patients)')
    lmf <- lmer(formula, pre_mean_nodelta)
    coefs <- coef(lmf)$patients
    slopes <- coefs$sessions
    pre_slope_data['preposts'] <- rep('Pre', 10)
    pre_slope_data['patients'] <- rownames(coefs)
    pre_slope_data[i] <- slopes
}

boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00430557 (tol = 0.002, component 1)”boundary (singular) fit: see ?isSingular


In [27]:
write.csv(pre_slope_data, 'LME_pre_slopes_NEW.csv')

In [9]:
# get Post slopes dataframe
for(i in names(post_mean_data)[4:28]) {
    formula = paste0(i,'~ sessions + (1 + sessions|patients)')
    lmf <- lmer(formula, post_mean_data)
    coefs <- coef(lmf)$patients
    slopes <- coefs$sessions
    post_slope_data['preposts'] <- rep('Post', 10)
    post_slope_data['patients'] <- rownames(coefs)
    post_slope_data[i] <- slopes
}

boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00398826 (tol = 0.002, component 1)”Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00249178 (tol = 0.002, component 1)”Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.021294 (tol = 0.002, component 1)”Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00208656 (tol = 0.002, component 1)”boundary (singular) fit: see ?isSingular
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00543867 (tol = 0.002, component 1

In [10]:
# get Diff slopes dataframe
for(i in names(diff_mean_data)[4:28]) {
    formula = paste0(i,'~ sessions + (1 + sessions|patients)')
    lmf <- lmer(formula, diff_mean_data)
    coefs <- coef(lmf)$patients
    slopes <- coefs$sessions
    diff_slope_data['preposts'] <- rep('Diff', 10)
    diff_slope_data['patients'] <- rownames(coefs)
    diff_slope_data[i] <- slopes
}

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.0053062 (tol = 0.002, component 1)”boundary (singular) fit: see ?isSingular
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.012937 (tol = 0.002, component 1)”Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00890197 (tol = 0.002, component 1)”boundary (singular) fit: see ?isSingular
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00249178 (tol = 0.002, component 1)”Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.021294 (tol = 0.002, component 1)”Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv

In [12]:
# save data in csv

slope_dat <- rbind(pre_slope_data, post_slope_data, diff_slope_data)
#write.csv(slope_dat, 'LME_slopes.csv')

In [27]:
formula = 'theta_pows ~ sessions + (1 + sessions|patients)'
lmf <- lmer(formula, pre_mean_data)
summary(lmf)

boundary (singular) fit: see ?isSingular


Linear mixed model fit by REML ['lmerMod']
Formula: theta_pows ~ sessions + (1 + sessions | patients)
   Data: pre_mean_data

REML criterion at convergence: 12.3

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-1.02044 -0.50878 -0.31323  0.06405  2.65640 

Random effects:
 Groups   Name        Variance  Std.Dev. Corr
 patients (Intercept) 0.0000000 0.0000       
          sessions    0.0001231 0.0111    NaN
 Residual             0.0641070 0.2532       
Number of obs: 22, groups:  patients, 10

Fixed effects:
            Estimate Std. Error t value
(Intercept)  0.18997    0.09293   2.044
sessions     0.03625    0.01425   2.544

Correlation of Fixed Effects:
         (Intr)
sessions -0.783
convergence code: 0
boundary (singular) fit: see ?isSingular


In [25]:
formula = 'chans_exps ~ sessions + QIDS + X5__Mood__sad_ + (1 + sessions|patients)'
lmf <- lmer(formula, pre_mean_data)
summary(lmf)

Linear mixed model fit by REML ['lmerMod']
Formula: chans_exps ~ sessions + QIDS + X5__Mood__sad_ + (1 + sessions |  
    patients)
   Data: pre_mean_data

REML criterion at convergence: 6.9

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-1.58674 -0.38514  0.05344  0.41238  1.26727 

Random effects:
 Groups   Name        Variance  Std.Dev. Corr 
 patients (Intercept) 0.0644526 0.25388       
          sessions    0.0008686 0.02947  -0.96
 Residual             0.0155877 0.12485       
Number of obs: 22, groups:  patients, 10

Fixed effects:
               Estimate Std. Error t value
(Intercept)     0.71904    0.23441   3.067
sessions        0.03124    0.01607   1.945
QIDS            0.02649    0.01341   1.976
X5__Mood__sad_ -0.06140    0.06344  -0.968

Correlation of Fixed Effects:
            (Intr) sessns QIDS  
sessions    -0.844              
QIDS        -0.683  0.304       
X5__Md__sd_ -0.054  0.289 -0.622

In [28]:
formula = 'QIDS ~ sessions + chans_exps + alphas + thetas + alpha_pows + theta_pows + (1 + sessions|patients)'
lmf <- lmer(formula, pre_mean_data)
summary(lmf)

Linear mixed model fit by REML ['lmerMod']
Formula: QIDS ~ sessions + chans_exps + alphas + thetas + alpha_pows +  
    theta_pows + (1 + sessions | patients)
   Data: pre_mean_data

REML criterion at convergence: 87.7

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-0.69167 -0.15634 -0.00026  0.24247  0.81847 

Random effects:
 Groups   Name        Variance Std.Dev. Corr 
 patients (Intercept) 6.2157   2.4931        
          sessions    0.4184   0.6468   -0.44
 Residual             1.5724   1.2540        
Number of obs: 21, groups:  patients, 10

Fixed effects:
            Estimate Std. Error t value
(Intercept)  12.0074    13.3590   0.899
sessions     -0.8144     0.2790  -2.919
chans_exps   -1.4976     5.1167  -0.293
alphas        0.5430     0.5356   1.014
thetas        0.7848     0.7838   1.001
alpha_pows   -1.9387     5.5297  -0.351
theta_pows   -2.2953     2.7247  -0.842

Correlation of Fixed Effects:
           (Intr) sessns chns_x alphas thetas alph_p
session

In [22]:
labels(pre_mean_data)

[[1]]
 [1] "1"  "2"  "3"  "4"  "5"  "6"  "7"  "8"  "9"  "10" "11" "12" "13" "14" "15"
[16] "16" "17" "18" "19" "20" "21" "22" "23"

[[2]]
 [1] "patients"                   "sessions"                  
 [3] "preposts"                   "chans_exps"                
 [5] "alphas"                     "alpha_pows"                
 [7] "thetas"                     "theta_pows"                
 [9] "QIDS"                       "X1__Sleep_onset_insomnia"  
[11] "X2__Mid_nocturnal_insomnia" "X3__Early_morning_insomnia"
[13] "X4__Hypersomnia"            "X1_4_Highest"              
[15] "X5__Mood__sad_"             "X7__Appetite__increased_"  
[17] "X8__Appetite__decreased_"   "X9__Weight__decrease_"     
[19] "X10__Weight__increase_"     "X7_10_Highest"             
[21] "X11__Concentration"         "X12__Outlook__self_"       
[23] "X13__Suicidal_ideation"     "X14__Involvement"          
[25] "X15__Energy"                "X16__Psychomotor_slowing"  
[27] "X17__Psychomotor_agitation" "X16_17_Highest"